In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
df=pd.read_csv('/content/breast_cancer.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [6]:
df.diagnosis.value_counts()/len(df)*100

,count
diagnosis,
B,62.741652
M,37.258348


In [7]:
df['diagnosis']=df['diagnosis'].map({'M':1,'B':0})

In [8]:
df.diagnosis.value_counts()

,count
diagnosis,
0,357
1,212


In [9]:
X=df.iloc[:,[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
y=df.diagnosis.values

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [12]:
len(X_train)

455

In [14]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
# RF classifier

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
y_pred=model.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,y_pred)*100)

94.73684210526315


In [18]:
cm=confusion_matrix(y_test,y_pred)
cm

array([[69,  2],
       [ 4, 39]])

In [19]:
cr=classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96        71
           1       0.95      0.91      0.93        43

    accuracy                           0.95       114
   macro avg       0.95      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114



In [20]:
# Manual HPO
n_estimators_list=[1,2,3,4,10,20,50,100]
for estim_list in n_estimators_list:
  model=RandomForestClassifier(n_estimators=estim_list)
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  result=accuracy_score(y_test,y_pred)*100
  print(estim_list)
  print(result)

1
85.08771929824562
2
92.98245614035088
3
93.85964912280701
4
96.49122807017544
10
93.85964912280701
20
96.49122807017544
50
95.6140350877193
100
95.6140350877193


In [22]:
leaf_size_list=[10,20,30,40,50]
for i in leaf_size_list:
  model=RandomForestClassifier(n_estimators=20,min_samples_leaf=i)
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  result=accuracy_score(y_test,y_pred)*100
  print(i)
  print(result)

10
94.73684210526315
20
92.10526315789474
30
94.73684210526315
40
92.98245614035088
50
96.49122807017544


# RANDOM SEARCH CV

In [23]:
# Random Search CV

from sklearn.model_selection import RandomizedSearchCV

n_estimators=[int(x) for x in np.linspace(start=100,stop=1000,num=10)]


In [25]:
max_depth = [int(x) for x in np.linspace(start=10, stop=100, num=10)]
min_samples_split = [2, 5, 10, 15,45,60,100]
bootstrap=[True,False]
min_samples_leaf=[2,5,15,20,50,80,100]


In [26]:
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap,
               'min_samples_leaf':min_samples_leaf}

On each iteration, the algorithm select random combinations of hyperparameters

In [28]:
rf=RandomForestClassifier()
rf_random=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,n_jobs=-1)
rf_random.fit(X_train,y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100],
                                        'min_samples_leaf': [2, 5, 15, 20, 50,
                                                             80, 100],
                                        'min_samples_split': [2, 5, 10, 15, 45,
                                                              60, 100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]})

In [29]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy=accuracy_score(test_labels,predictions)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

In [30]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Model Performance
Accuracy = 0.94%.


In [31]:
best_random=rf_random.best_estimator_
print(best_random)

RandomForestClassifier(bootstrap=False, max_depth=60, min_samples_leaf=2,
                       n_estimators=900)


In [32]:
random_accuracy=evaluate(best_random,X_test,y_test)

Model Performance
Accuracy = 0.96%.


# GRID SEARCH CV

In [33]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [70, 80, 90, 100],
    'max_features': [2,3,4,5,6,7],
    'min_samples_leaf': [1,2,4,6,7],
    'min_samples_split': [2, 5, 10, 15],
    'n_estimators': [100, 200, 300, 400, 480,560,880,1000]
}

rf_gs=RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf_gs, param_grid = param_grid, cv=3,n_jobs=-1)

In [34]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [ ]:
best_grid=grid_search.best_estimator_
grid_accuracy=evaluate(best_grid,X_test,y_test)